Владимир Бадло. Задание 2.

In [1]:
from scipy import stats
import numpy
import matplotlib.pyplot as pyplot
from statsmodels.stats.weightstats import ttest_ind

Разработчики Петр и Вася занимаются разработкой функционала для звонков через приложение Авито. Петр занимается Android версией приложения, а Вася занимается iOS. Совсем недавно был обнаружен странный баг, который мешал пользователям звонить через приложение - они постоянно теряли связь при разговоре и нужно было звонить заново. Петя и Вася "починили" баг и обновили клиенты приложений, чтобы пользователи могли общаться спокойно.

Известно, что доля звонков через Android составляет ровно 70% всех звонков. В то время как через iOS звонят оставшиеся 30%.
Также было известно что в версии А средняя продолжительность звонка составляла 50 секунд со средним отклонением 10 секунд.

Основная метрика, на которую смотрит команда при разработке продукта - средняя продолжительность звонка.

После обновления ожидалось, что средняя продолжительность звонков вырастет до 100 секунд с тем же средним отклонением.

Однако, после запуска версии В средняя продолжительность звонка увеличилась лишь до 58 секунд. Этот результат устроил команду, однако аналитик задумался почему так произошло и решил "изучить" данные подробнее.

Для этого он решил провести два теста, T-test и тест Манна-Уитни чтобы сравнить версии приложения.


Пусть генеральная совокупность А - продолжительность звонков пользователей в приложении версии А ($\xi_A$) . Генеральная совокупность В - продолжительность звонков в приложении версии В ($\xi_B$).

In [56]:
# import scipy.stats as stats
import numpy

class version_A(stats.rv_continuous): # rv_discrete
    "Exponential distribution also having a small chance of a large object drawn from a uniform distribution"
    
    def _pdf(self, x, android_share):
        # NB: arguments can by numpy arrays  
        
        return (
            (1 - android_share)
            *
            stats.norm(loc=50, scale=10).pdf(x)
            
            +
            
            android_share
            *
            stats.norm(loc=50, scale=10).pdf(x)            
        )
        
    
    # rvs can be calculated using defined pdf, but it will work faster if defined explicitly
    def _rvs(self, android_share, size, random_state):
        uniform = stats.uniform(loc=0, scale=1).rvs(size)

        return numpy.where(
            uniform <= android_share, 
            stats.norm(loc=50, scale=10).rvs(size), 
            stats.norm(loc=50, scale=10).rvs(size)
        ) 
    
        
calls_dist_A = version_A(name="calls_distribution")

class version_B(stats.rv_continuous): # rv_discrete
    "Exponential distribution also having a small chance of a large object drawn from a uniform distribution"
    
    def _pdf(self, x, android_share):
        # NB: arguments can by numpy arrays  
        
        return (
            (1 - android_share)
            *
            stats.norm(loc=100, scale=10).pdf(x)
            
            +
            
            android_share
            *
            stats.norm(loc=40, scale=10).pdf(x)            
        )
        
    
    # rvs can be calculated using defined pdf, but it will work faster if defined explicitly
    def _rvs(self, android_share, size, random_state):
        uniform = stats.uniform(loc=0, scale=1).rvs(size)

        return numpy.where(
            uniform <= android_share, 
            stats.norm(loc=40, scale=10).rvs(size), 
            stats.norm(loc=100, scale=10).rvs(size)
        ) 
    
        
calls_dist_B = version_B(name="calls_distribution")


In [44]:
calls_A = calls_dist_A(android_share=0.7)
calls_B = calls_dist_B(android_share=0.7)

In [45]:
sampleA = list(calls_A.rvs(size=300))
sampleB = list(calls_B.rvs(size=300))

С прошлой версии приложения аналитик помнил, что средняя продолжительность звонка имеет нормальное распределение. Поэтому первое, что пришло в голову - проверить разницу матожиданий через T-test.

In [51]:
ttest_ind(sampleB, sampleA, alternative='larger')

(4.042131021518202, 2.9944505477505386e-05, 598.0)

Результат оказался стат значимым. Среднее выборки В оказалось выше среднего выборки А.    

Тем не менее аналитик решил "подстраховаться" и перепроверить результат на непараметрическом тесте Манна-Уитни. 

Аналитик вспомнил, что тест Манна-Уитни проверяет немного другие гипотезы о распределнии:

$H_0$: cлучайные величины $\xi_A$ и $\xi_B$ имеют одно распределение

$H_1: \mathbb{P}(\xi_A < \xi_B) > 0.5$

In [52]:
stats.mannwhitneyu(sampleB, sampleA, alternative='greater')

MannwhitneyuResult(statistic=39861.0, pvalue=0.9922561010550056)

Результат удивил - тест Манна-Уитни не дает права отвергнуть нулевую гипотезу.

In [54]:
stats.mannwhitneyu(sampleB, sampleA, alternative='less')

MannwhitneyuResult(statistic=39861.0, pvalue=0.0077539377584963315)

Согласно тесту Манна-Уитни распределение выборки B "лежит левее" распределения выборки А. Тем не менее матожидание этих выборок расположены наоборот.

Тогда Аналитик решил посмотреть на данные в разрезе платформ и выяснил, что теперь пользователи Android в среднем имеют продолжительность разговора  не 50 секунд, а 40 секунд. В то время как для iOS приложения, средняя продолжительность разговора действительно увеличилась до 100 секунд как и ожидалось. 
Из-за того, что доля звоноков через Android составляет 70%, общая генеральная совокупность В действительно "сдвинулась" влево относительно генеральной совокупности А. Поэтому, несмотря на увеличение математического ожидания, тест Манна-Уитни показал такой неожиданный результат. Новое распределение генеральной совокупности В стало бимодальным и теперь его нельзя сравнивать с распределением А с помощью теста Стьюдента.